In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/nfshome/jc9033/.conda/envs/py3/bin/python3.7'

from pyspark import SparkContext
sc = SparkContext('local', 'pyspark')

from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [2]:
sc

<SparkContext master=local appName=pyspark>

In [3]:
def mapper1(partitionID, rows):
    if partitionID==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        yield((fields[4],1))

In [ ]:
%%time
i = "2015"
path = "data_tickets/tickets_" + i + ".csv"
rdd = sc.textFile(path)\
         .mapPartitionsWithIndex(mapper1)\
         .groupByKey().mapValues(sum)         

In [ ]:
%%time
rdd.take(5)

# Cleaning & Integrating

In [7]:
%%time
for i in range(2014,2020):
    print(i)
    path = "data_tickets/tickets_" + str(i) + ".csv"
    if i == 2014:
        rdd = sc.textFile(path)\
                .mapPartitionsWithIndex(mapper1)\
                .groupByKey().mapValues(sum) 
    else:
        temp = sc.textFile(path)\
                 .mapPartitionsWithIndex(mapper1)\
                 .groupByKey().mapValues(sum) 
        rdd = rdd.union(temp)
    rdd.take(5)

2014
2015
2016
2017
2018
2019
CPU times: user 148 ms, sys: 44 ms, total: 192 ms
Wall time: 5min 31s


### Output CSV

In [8]:
%%time
from pyspark import SparkContext
df = spark.createDataFrame(rdd, ['date', 'count'])

CPU times: user 200 ms, sys: 144 ms, total: 344 ms
Wall time: 3.29 s


In [9]:
df.show()

+----------+-----+
|      date|count|
+----------+-----+
|02/11/2000|    5|
|04/30/2000|    1|
|02/11/2001|    2|
|02/26/2001|    1|
|10/03/2003|    2|
|03/21/2004|    1|
|05/01/2004|    2|
|05/11/2004|    1|
|09/04/2004|    1|
|10/07/2010|    1|
|01/08/2011|    1|
|02/28/2011|    1|
|09/02/2011|    3|
|02/28/2012|    2|
|05/11/2012|    6|
|09/04/2012|    1|
|10/13/2012|   10|
|02/07/2013|    8|
|02/26/2013|   13|
|05/05/2013|   10|
+----------+-----+
only showing top 20 rows



In [10]:
%%time
df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('output_tickets')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 16.6 s


###  Check file

In [11]:
filename = ! ls output_tickets | grep .csv

In [12]:
import pandas as pd
pddf = pd.read_csv('output_tickets/'+filename[0]).fillna(0)
print(pddf.shape)

(11325, 2)


In [13]:
pddf.head()

,date,count
0,02/11/2000,5
1,04/30/2000,1
2,02/11/2001,2
3,02/26/2001,1
4,10/03/2003,2


In [14]:
pddf.tail()

,date,count
11320,11/25/2019,5
11321,12/06/2019,5
11322,01/08/2018,25
11323,12/21/2019,8
11324,12/22/2000,1
